In [ ]:
using Plots
using Images
using ImageView
using Statistics
using Colors
using Serialization
using FFTW

In [ ]:
imagename = "IMG-HH"

## Schema

In [ ]:
function bytesToString(file, start, stop)
    len = stop-start+1
    seek(file,start-1)
    String(read(file,len))
end

function bytesToUInt32(file, start, stop)
    #TODO check if range matches size
    seek(file, start-1)
    num = try
        read(file,UInt32)
    catch
        0
    end
    Int32(ntoh(num))
end

function bytesToUInt16(file, start, stop)
    #TODO check if range matches size
    seek(file, start-1)
    Int16(ntoh(read(file,UInt16)))
end

function bytesToArray(file, start, stop)
    len = stop-start+1
    seek(file, start-1)
    read(file,len)
end

#Image File Descriptor
imageFileDescrictorScheme = [    
        ("length", (9, 12), bytesToUInt32),
        ("numSignals", (181, 186), bytesToString),
        ("sarDataBytes", (187, 192), bytesToString),
        ("bitsPerSample", (217, 220), bytesToString),
        ("samplesPerPixel", (221, 224), bytesToString), #could be 1 or 2 (IQ or A)
        #245-272 - borders and interleaving
        ("physicalRecordsPerLine", (273, 274), bytesToString),
        #281-400 - unexplored
        ("sarDataFormat", (401, 428), bytesToString),
        ("sarDataFormatTypeCode", (429,432), bytesToString)
        
        ]

signalDataRecordScheme = [
        ("recordSequenceNumber", (1,4), bytesToUInt32),
        ("lengthOfRecord", (9,12), bytesToUInt32),
        ("sarImageDatalineNumber", (13, 16), bytesToUInt32),
        ("pixelCount", (25, 28), bytesToUInt32),
        #33-56 - instrument settings
        #57-84 - chirp characteristics
        ("PRF (mHz)", (57,60), bytesToUInt32),
        ("Chirp type", (67,68), bytesToUInt16),    #0 for linear
        
        #93-388 - platform info - important!!!
        ("Valid", (97,100),bytesToUInt32),
        ("Electronic Squint 1",(101,104),bytesToUInt32),
        ("Electronic Squint 2",(109,112),bytesToUInt32),
        ("Mechanical Squint",(113,116),bytesToUInt32),
        ("FirstSampleRange (m)",(117,120),bytesToUInt32),
        ("Platform Altitude (m)",(141,144),bytesToUInt32),
        ("Platform Ground Speed (cm/s)",(145,148),bytesToUInt32),
        ("signalData", (413, 10800), bytesToArray)
        ]

datasetSummaryRecordScheme = [
    ("wavelength", (501,516), bytesToString),
    ("chirpType", (519,534), bytesToString),
    ("coeffs",(534,694),bytesToString),
    ("samplingRate",(711,726), bytesToString),
    ("pulseLength",(743,758),bytesToString),      #us
    ("PRF", (935,950), bytesToString),            #mHz
    ("doppler", (1415,1654), bytesToString)
]

function parseFile(file,scheme,offset=0)
    keyMap = Dict([])
    for i=1:length(scheme)
        keyMap[scheme[i][1]] = i  #make dictionary mapping field names to indices
    end
    results = []
    for (name, (start, stop), f) in scheme
        len  = stop-start+1
        push!(results, f(file,start+offset,stop+offset))
    end
    return (key=keyMap, fields=results)
end

## File Processing
This stage converts a `.0__A` file downloaded from ASF Vertex into a serialized Julia object in a `.ser` file. Can be skipped if you've already produced a `.ser` file for the SAR image you want to process.

In [ ]:
file = open("Cape/$imagename.0__A")

imageDescriptor = parseFile(file,imageFileDescrictorScheme)
sarDataBytes    = parse(Int64,imageDescriptor.fields[imageDescriptor.key["sarDataBytes"]])
numSignals      = parse(Int64,imageDescriptor.fields[imageDescriptor.key["numSignals"]])

signalRecords = [ ]

for i = 1:(numSignals-1)
    parsedRecord = parseFile(file,signalDataRecordScheme,720+sarDataBytes*i)
    signal = parsedRecord.fields[parsedRecord.key["signalData"]]
    
    #TODO  - is 15 or 16 better?? compute mean of signals?
    #TODO  - DEAL WITH VALUES > 0x1f!!
    signal = map(x->min(0x1f,x), signal)
    
    signal = Int8.(signal)-Int8.(15*ones(size(signal)))
    I = signal[1:2:length(signal)]
    Q = signal[2:2:length(signal)]

    parsedRecord.key["I"] = length(parsedRecord.fields)+1;
    parsedRecord.key["Q"] = length(parsedRecord.fields)+2;
    push!(parsedRecord.fields, I)
    push!(parsedRecord.fields, Q)
    
    parsedRecord.fields[parsedRecord.key["signalData"]] = []  #remove original signalData from record!
    
    push!(signalRecords,parsedRecord)
    if(i%10000==0)
        print(i)
        print("   ")
        print(Base.summarysize(parsedRecord))
        print("B    ")
        print(Base.summarysize(signalRecords)/1e6)
        println(" MB")
    end
end
print(Base.summarysize(signalRecords)/1e6)
println(" MB")

close(file)

# save signalRecords for later use
Serialization.serialize(open("Cape/$imagename-signal-records.ser","w"),signalRecords)
signalRecords = [];  # free up the memory of the signalRecords object - it'll be read back in later

## Chirp Finding
This stage produces a chirp signal that will deconvolve the IQ samples stored in the L1.0 file read in above. Cannot be skipped!

In [ ]:
#process header
file = open("Cape/LED.0__A")
rec = parseFile(file,datasetSummaryRecordScheme,720)
#rec.fields[rec.key["PRF"]]

f = parse(Float64,split(rec.fields[rec.key["coeffs"]])[1])
fdot = -parse(Float64,split(rec.fields[rec.key["coeffs"]])[2])
sampleRate = parse(Float64,rec.fields[rec.key["samplingRate"]])*1e6     #Hz
pulseLength = parse(Float64,rec.fields[rec.key["pulseLength"]])*1e-6   #s
PRF = parse(Float64,rec.fields[rec.key["PRF"]])/1000
pulseSamples = Integer(floor(pulseLength*sampleRate))
#pulseSamples = 400
print("Max IF freq: ")
println(fdot*pulseLength)

Sif(t) = exp(pi*im*fdot*t^2)

t = 1/sampleRate* (range(1, stop = pulseSamples) |> collect)

t = t.-maximum(t)/2

sig = Sif.(t)/sqrt(pulseSamples)

rangeCells = 5000

chirpI = vcat(real(sig),zeros(rangeCells))
chirpQ = vcat(imag(sig),zeros(rangeCells))
chirp = Complex{Float32}.(chirpI,chirpQ)

chirpFFT = fft(chirp)

print("Ready")

In [ ]:
R0 = 848665     #m  
altitude = 628000 #m, nominal
c = 3e8

a = (6371+628)*1000.0  #m            #https://www.eorc.jaxa.jp/ALOS-2/en/about/overview.htm
G = 6.67408e-11
Me= 5.97219e24       #kg
P = sqrt(4*pi^2/(G*Me)*a^3)
print("Period: ")
print(P/3600)
println(" h")
vorbital = 2pi*a/P            #These are terrible assumptions!!!!!!!
vorbital = 7593
println("Orbital Velocity: ",vorbital," m/s")

wavelength = parse(Float32,rec.fields[rec.key[  "wavelength" ]]) #m 
#antenna length
La = 8.9 #m

#beamwidth
bw = 0.886*wavelength/La

In [ ]:
nadirAngle = acos(altitude/R0)
println(nadirAngle*180/pi)
RD = 1/2*c*5000/16000000
rangeCellLength = 1/5000 * (sqrt((R0+RD)^2-altitude^2)-sqrt(R0^2-altitude^2))

## Image Forming
This stage converts the `...-signal-records.ser` object produced above into an early-stage image. The output of these blocks is a range-compressed complex image in a `.rcc` file.

In [ ]:
showRaw = true;

In [ ]:
# load serialized version of signalRecords if not loaded from file above
signalRecords = Serialization.deserialize(open("Cape/$imagename-signal-records.ser","r"))
print("Loaded signalRecords")

In [ ]:
#sub image formation:
sampleNum = 5000 #how many samples of each echo to keep
echoNum = 35000  #how many echos to keep
echostart = 1

# smallSignals is matrix containing a subset (defined by sampleNum and echoNum)
# of the echo signals in signalRecords
smallSignals = zeros(Complex{Float16},sampleNum,echoNum)

#This is soooooo much faster than hcatS!!!!
#https://stackoverflow.com/questions/38308053/julia-how-to-fill-a-matrix-row-by-row-in-julia
for i = echostart+1:echostart+echoNum
    line = signalRecords[i]
    I = Float16.(line.fields[line.key["I"]])
    Q = Float16.(line.fields[line.key["Q"]])
    
    smallSignals[:,i-echostart] = Complex.(I[1:sampleNum],Q[1:sampleNum])
end

signalRecords = []  #free up signalRecords - now using smallSignals

print("Done")

In [ ]:
# TODO: serialize smallSignals?
#Serialization.serialize(open("/home/thatcher/Documents/12.421/smallSignalsBigger.ser","w"),smallSignals)

In [ ]:
shape = size(smallSignals)
rawMagnitude = abs.(view(smallSignals,1:10:shape[1],1:40:shape[2]))';
rawMagnitude = reverse(rawMagnitude,dims=1)
imshow(rawMagnitude);
# TODO: might want to extend the width of the image by pulseSamples before downsizing,
# as was done in the original version. See snippet below:
# vcat(zeros(Complex{Float16},pulseSamples),smallSignals[:,i])

In [ ]:
# TODO: shorten this down. No need to append line-by-line like this

shape = size(smallSignals)

cimg = zeros(Complex{Float16},(shape[1]+pulseSamples,shape[2]))

for i = 1:shape[2]
    echo = vcat(zeros(Complex{Float16},pulseSamples),smallSignals[:,i])
    
    ####### Range Compression
    signalFFT = fft(Complex{Float32}.(echo))
    
    crossCorrelated = AbstractFFTs.ifft(conj.(chirpFFT).*signalFFT)
    ####### End Range Compression
    
    cimg[:, i] = Complex{Float16}.(crossCorrelated)
    if i%10000 == 1
        println(i)
    end
end

cimg= cimg'
smallSignals = []  # drop small signals - using cimg now
GC.gc()

print("Done")

In [ ]:
shape = size(cimg)
deconvolvedMagnitude = abs.(view(cimg,1:40:shape[1],1:10:shape[2]));
deconvolvedMagnitude = reverse(deconvolvedMagnitude,dims=1)
imshow(deconvolvedMagnitude);

In [ ]:
Serialization.serialize(open("$imagename.rcc","w"),cimg)
cimg = [];
GC.gc();

## Loading Range Corrected Complex Image File
Can start here if a range-corrected complex image is already available from IMG-HH.rcc or similar.

In [ ]:
cimg = Serialization.deserialize(open("$imagename.rcc","r"))
print("Loaded")

### Range Cell Migration Correction
Going to using the Range Doppler Algorithm. Take range compressed data, fourier transform along each line of constant range, then interpolate by a azimuth-frequency-dependent amount to correct for range cell migration.

Could image at this point, but probably won't look different. 

After that, do azimuth compression as usual. To be efficient, don't apply IFFT to RCM corrected data and instead use that direction in the azimuth convolutions.

Range shift at each frequency is:
$$\Delta R(f_n) = \frac{\lambda^2 R_0 f_n^2}{8 V_r^2}$$
Where $R_0$ is the distance of closest approach, $V_r$ is the effective radar velocity (Cummings and Wong pg. 235)


In [ ]:
# perform RCMC in-place on cimg
# TODO: is it possible to perform all FFTs at once? 
#   there might be a dims argument to fft for matrices

Vr = vorbital

for i = 1:size(cimg)[2]
    line = Complex{Float32}.(cimg[:,i])

    ####### Fourier Transforming
    cimg[:,i] = fft(line)
    if i%1000 == 1
        print("#")
    end
end
println("\nDone")

In [ ]:
# show fourier transformed cimg if desired
# with the correct contrast, the curves that will be corrected
# by range cell migration should be visible
#
# this block can be run again after the following block to 
# show the effect of Range Cell Migration Correction
shape = size(cimg)
rccft = (abs.(view(cimg,
                1:100:shape[1],
                3600+54:1:3600+473)))
imshow(rccft);

In [ ]:
#now we want to shift each frequency in range space, so make each frequency bin a column for speed
cimg = cimg'
shape = size(cimg)
c = 3e8
slantRes = 1/2*c/sampleRate

for i = 1:shape[2]
    n = i
    #the "highest frequencies" are actually the negative frequencies aliased up!
    if n>shape[2]/2
        n = shape[2]-i
    end
    fn = (n-1)/shape[2]*PRF    #check this but pretty sure
    
    #range migration distance in meters
    ΔR = wavelength^2*R0*fn^2/(8*Vr^2)
    cellshift = Integer(round(ΔR/slantRes))
    
    #interpolation
    #NEAREST NEIGHBOR - bad!
    cimg[:,i] = vcat(cimg[cellshift+1:shape[1],i],zeros(Complex{Float64},cellshift))
    
    if n%10000==0
        print(n)
        print("  ")
        println(cellshift)
    end
end
cimg = cimg'
println("Done")

### Azimuth Compression!!
need PRF, altitude, initial range, ground speed, wavelength
$$R(s) = \sqrt{R_0^2+s^2v^2} = R_0+\dot{R}_0s+\frac{1}{2}\ddot{R}_0s^2$$
$$C(s) = e^{i \frac{4\pi}{\lambda} R(s)}$$ (4pi comes from: phase shift due to distance is 2pi, but you go there and back so phase shift is doubled)
Sample $C(s)$ at $s=n/PRF$

For zero doppler shift (kinda naive case but fine), $\dot{R}_0 = 0$ and $\ddot{R}_0 = \frac{v^2}{R_0}$

**Note:** this all only works for the first pixels!! Need to correct $R_0$ as we move out from the ground track.

In [ ]:
theta(s,R) = atan(vorbital*s/R)

#one way beam pattern:
p(a) = sinc(a*La/wavelength)
w(s,R) = p(theta(s,R))^2

Rc = R0+10000

R(s) = Rc - 1/2*vorbital^2/Rc*s^2
C(s) = exp(-4pi*im/wavelength*R(s))*w(s,Rc)

width = 200
s = 1/PRF*(range(-width, stop = width) |> collect)

sig = C.(s)/sqrt(width)

# TODO this is a weird way to make azimuth...
azimuthI = vcat(real(sig),zeros(size(cimg)[1]-length(sig)))
azimuthQ = vcat(imag(sig),zeros(size(cimg)[1]-length(sig)))
azimuth = Complex{Float32}.(azimuthI,azimuthQ)

complexAzimuthFFT = fft(azimuth)

print("Ready")

In [ ]:

for i = 1:size(cimg)[2]
    line = Complex{Float32}.(cimg[:,i])
    
    ####### Azimuth Compression
    
    #lineFFT = fft(line)
    lineFFT = cimg[:,i]
    #lineFFT = fft(Complex{Float32}.(cimg[:,i]))
    crossCorrelated = AbstractFFTs.ifft(conj.(complexAzimuthFFT).*lineFFT)
    
    ####### End Azimuth Compression
    
    
    #complex = Complex.(I,Q)
    result = abs.( crossCorrelated )
    cimg[:,i] = result
    if i%1000 == 1
        print("#")
    end
end

shape = size(cimg)
azcompmag = abs.(view(cimg,1:16:shape[1],1:4:shape[2]));
azcompmag = reverse(azcompmag,dims=1)
imshow(azcompmag);

In [ ]:
shape = size(cimg)
azcompmag = abs.(view(cimg,1:16:shape[1],1:4:shape[2]));
azcompmag = reverse(azcompmag,dims=1)
imshow(log.(azcompmag));

In [ ]:
# save range and azimuth compressed file as a "single-look complex"
Serialization.serialize(open("$imagename.slc","w"),azcomp)

## Images

In [ ]:
# compare the width and height of a pixel (cell)
# to get an approximate scaling for image formation
println("Azimuth:     ", round(vorbital/PRF, digits=3), " m")
println("Range Cell: ",round(rangeCellLength, digits=3), " m")
println("Aspect Ratio: ",round(rangeCellLength/(vorbital/PRF),digits=1))

In [ ]:
#magnitude image formation
scaling = 4
shape = size(azcomp)
#raw = abs.(view(transpose(img),
#                1:scaling:shape[1],
#                1:scaling:shape[2]))
#=
rangecompmag = (abs.(view(cimg,
                1:scaling:shape[1],
                1:scaling:shape[2])))
=#
#rcmcftmag = (abs.(view(rcmc,
#                1:scaling:5000,
#                1:scaling:8000)))
azcompmag = (abs.(view(azcomp,
                #1:Integer(round(scaling*rangeCellLength/azimuthCellLength)):shape[1],
                1:Integer(round(scaling*     4     )):shape[1],
                1:scaling:shape[2])))
#=
azcompmag = min.(60,abs.(view(azcomp,
                7000:scaling*4:11000,
                1750:scaling:2750)))
=#
#reduced = zeros(pixelsPerStrip)
#for c=1:(length(reduced)-2)                 #TODO why is -2 here??
#    reduced[c] = sum(mag[c*pixelStep:(c+1)*pixelStep])/pixelStep
#end
print("Ready")

In [ ]:
imshow(log.(azcompmag))

In [ ]:
imshow(azcompmag)

## Raw Magnitude Image Notes:
There are a few interesting features in the echo magnitude images: one is very long saturated strips that fill out most of an entire signal data record. This might have to be removed.... not sure why they are present at all.

The other feature is short dashes that are ~67 pixels long in the vertical direction. I suspect the pulse length is 67 sampling intervals long.

In [ ]:
@which varinfo()

In [ ]:
using Markdown
function myvarinfo(m::Module=Main, minsize::Number=0, pattern::Regex=r"")
    rows =
        Any[ let value = getfield(m, v)
                 Any[string(v),
                     (value===Base || value===Main || value===Core ? "" : Base.format_bytes(Base.summarysize(value))),
                     summary(value)]
             end
             for v in sort!(names(m)) if isdefined(m, v) && 
                    occursin(pattern, string(v)) && 
                    Base.summarysize(getfield(m, v)) >= minsize]

    pushfirst!(rows, Any["name", "size", "summary"])

    return Markdown.MD(Any[Markdown.Table(rows, Symbol[:l, :r, :l])])
end

In [ ]:
myvarinfo(Main, 1000)